<a href="https://colab.research.google.com/github/RobinSmits/OBSDeVelduil-AI-Demo/blob/master/AI_Demo_Natural_Language_Processing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Demo - Natural Language Processing 'Is de film recensie positief of negatief?'

Hoi! Welkom bij deze demo. 

In het komende uur gaan we in sneltreinvaart kijken hoe we een computer tekst kunnen leren herkennen. Om specifiek te zijn we gaan de computer leren om op basis van een film recensie aan te geven of deze positief of negatief is.

De stappen die we gaan volgen zijn een eenvoudige samenvatting van een zeer uitgebreidde handleiding over dit onderwerp bij Tensorflow. Mocht je de uitgebreide code dus een keer willen doorwerken dan kun je deze vinden op onderstaande link:
https://www.tensorflow.org/tutorials/keras/text_classification

# Wat gaan we de computer leren?

We hebben om de computer tekst te leren herkennen een dataset met veel tekst nodig. We gaan de IMDB Dataset hiervoor gebruiken.

De IMDB dataset bevat de tekst van 50000 film recensies. Er zijn film recensies die negatief zijn of die positief zijn.

We gaan de computer trainen met 25000 film recensies en vervolgens gaan we kijken of de computer voor een 2de set (de testset) van 25000 recensies voor elke recensie kan voorspellen of deze positief of negatief is.

Om de computer iets te leren moeten we een klein computer programma schrijven. Dit computer programma noemen we het 'model'.
Zodra we de computer daadwerkelijk gaan laten leren zijn we het 'model' aan het 'trainen'.



# De start

We gaan eerst een aantal Python programma's laden en in gebruik nemen. Deze hebben we nodig om het machine learning model op te zetten en de data te kunnen gebruiken.

In [ ]:
# Tensorflow is een Artificial Intelligence en Machine Learning programma van Google (Meer weten? https://www.tensorflow.org/learn )
import tensorflow as tf
from tensorflow import keras

# Met de tensorflow_datasets kunnen we de IMDB dataset straks ophalen
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Numpy is voor diverse berekeningen. Die hebben we straks ook nodig.
import numpy as np

# Data verzamelen

We gaan de data voor de film recensies downloaden via de eerdere programma's.

De dataset is al helemaal voor ons voorbereid. Het belangrijkste is dat alle woorden zijn omgezet naar nummers. En elk nummer verwijst naar het woord in wat ze een vocabulary noemen.

Een computer kan niet rechtstreeks met de tekst werken...maar als we de tekst nou op een slimme wijze omzetten naar nummers..dan kan het wel.

In [ ]:
# We laden de IMDB Dataset
(train_data, test_data), info = tfds.load(
    # Use the version pre-encoded with an ~8k vocabulary.
    'imdb_reviews/subwords8k', 
    # Return the train/test datasets as a tuple.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Return (example, label) pairs from the dataset (instead of a dictionary).
    as_supervised=True,
    # Also return the `info` structure. 
    with_info=True)


We kunnen heel makkelijk zien hoe de tekst word omgezet naar een aantal nummers. 

In [ ]:
# We maken een vertaler aan...die de tekst omzet naar nummers
encoder = info.features['text'].encoder

# En we zetten een voorbeeld zin om
voorbeeld = 'Hallo allemaal. Welkom bij de tekst demo.'
voorbeeld_in_nummers = encoder.encode(voorbeeld)
print(f'Voorbeeld in nummers: {voorbeeld_in_nummers}')

Of nu met de losse woorden....

In [ ]:
print(f'Hallo = {encoder.encode("Hallo")}')
print(f'Hallo = {encoder.encode("Hallo ")}')
print(f'allemaal. = {encoder.encode("allemaal.")}')

En wat we zien is dat 1 woord toch meerdere nummers kan opleveren. Wat de encoder doet is bijvoorbeeld het einde van de zin of een spatie aangeven. Ook worden woorden wel eens opgesplitst.

Kijk maar :-)

In [ ]:
print(f'4313 = "{encoder.decode([4313])}"')
print(f'8040 = "{encoder.decode([8040])}"')
print(f'222  = "{encoder.decode([222])}"')

Laten we nog even een 2 tal voorbeelden uit de IMDB dataset bekijken.

Voor elk voorbeeld laten we de volledig tekst in nummers, de tekst zelf en het label.

In [ ]:
for train_example, train_label in train_data.take(2):
  print('Tekst nummers:', train_example.numpy())
  print('Tekst:', encoder.decode(train_example))
  print('Label:', train_label.numpy())

We doen nu even de laatste paar stappen om de data voor te bereiden.

We bereidden de trainings data voor waarmee we het model trainen en de tekst leren te herkennen.

En we bereidden de test data voor waarmee we het model straks testen en kijken hoe goed we voor onbekende film recensies kunen voorspellen of deze positief of negatief zijn.

In [ ]:
BUFFER_SIZE = 1000
train_batches = (train_data.shuffle(BUFFER_SIZE).padded_batch(32))
test_batches = (test_data.padded_batch(32))

We gaan nu een simpel model opzetten met behulp van een programma genaamd 'Tensorflow'. Met Tensorflow kun je eenvoudige modellen zoals we hierna gaan doen maken tot de meest complexe AI systemen denkbaar.

Wat we in het model stoppen zijn de reeksen met cijfers die de woorden voorstellen. Wat we voorspellen is of de recensie negatief (0) of positief (1) is.

In [ ]:
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1, activation='sigmoid')])

We kunnen heel simpel bekijken hoe ons model er uitziet.

In [ ]:
model.summary()

# Model Trainen

We gaan nu ons model trainen door het 10 keer alle film recensies te laten 'lezen'. En elke keer wordt erbij getoond (met het label!) of deze recensie positief of negatief was.

Het model gaat dan leren bijvoorbeeld of er bepaalde woorden of zinnen gebruikt worden voor een positieve of negatieve recensie.

In [ ]:
# Deze regels zijn nodig om het model compleet te maken..
loss = tf.keras.losses.BinaryCrossentropy(from_logits = False)

# Als je zelf wat wilt spelen met hoe het model leert...
# Je kan de learning_rate bijvoorbeel groter of kleiner maken.
# Als je hem kleiner maakt duurt het leren langer.
# Als je hem groter maakt leert hij sneller en het kan zelfs zijn dat hij slechter leert.
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

# We 'compileren' ==> 'gereed maken' het model met zijn optimizer, loss en metrics
model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])

# Hier starten we het trainen van het model
model.fit(train_batches,
          epochs = 10,
          validation_data = test_batches)

We hebben nu ons model getrained. De getallen aan de rechterkant ('loss' en 'accuracy') geven aan het goed het model is getrained.

De 'loss' willen we altijd zo laag mogelijk (richting 0 krijgen) en de 'accuracy' zo hoog mogelijk (richting 1)

We kunnen ons model nu testen. Laten we eens kijken hoe goed (of misschien wel slecht??) het model is.

In [ ]:
# Test het model
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

We zien dat ons model ongeveer rond de 85% a 86% juist een voorspelling maakt.

Ik ben benieuwd of we nog hoger kunnen scoren...laten we nog een 2de model proberen wat net iets beter kan leren.

In [ ]:
# Model 2
model2 = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(32, activation='relu'),
  keras.layers.Dense(1, activation='sigmoid')])

# Samenvatting van Model 2
model2.summary()

# Deze regels zijn nodig om het model compleet te maken..
loss = tf.keras.losses.BinaryCrossentropy(from_logits = False)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005)

# We 'compileren' ==> 'gereed maken' het model met zijn optimizer, loss en metrics
model2.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])

# Hier starten we het trainen van het 2de model
model2.fit(train_batches,
          epochs = 10,
          validation_data = test_batches)

En als we nu weer ons model testen...zullen we zien dat het net iets beter scored dan het eerste model.

Doordat we het model hebben uitgebreid kan het als het ware meer dingen leren.

In [ ]:
# Test het Model 2
loss, accuracy = model2.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
# Los voorbeeld tekst.
voorbeeld_in_nummers = encoder.encode('This was probably the most terrible movie.. a complete disaster')
print(model2.predict([voorbeeld_in_nummers]))